# Métodos de Apoio à Decisão Multicritério 

## Integral de Choquet e metodos não-supervisionados 

In [50]:
#Bibliotecas utilizadas
import numpy as np
import itertools
import operator as op
from functools import reduce
from scipy.special import bernoulli #Biblioteca para importar os Números de Bernoulli.
import math as math
import pandas as pd

In [2]:
# Definição das funções

# Função para o cálculo da integral de Choquet
def intChoquet(X, capacidade):
    # Etapa 1) Criando uma matriz com todas as coligações em ordem lexicográfica: 
    n = len(X[0])  # Número de critérios
    p = np.arange(1,
                  n + 1)  # Vetor 1,2...n para ser utilizado como suporte para obter todas as combinações em ordem 
    # lexicográfica 
    S = np.concatenate(
        [np.pad(np.array(list(itertools.combinations(p, r))), ((0, 0), (0, n - r)), 'constant', constant_values=0) for r
         in range(0, n + 1)], axis=0)
    # ^ Matriz que contem todas as combinações em ordem lexicográfica   

    # Etapa 2) Processo de ordenamento das alternativas em ordem crescente e coletando e ordenando seus índices 
    X_aumentado = np.pad(X, ((0, 0), (0, 1)), 'constant',
                         constant_values=0)  # Aumenta a matriz de decisão e acrescenta uma coluna de zeros
    X_new = np.sort(X_aumentado)  # Ordenamento da matriz aumentada
    X_indice = np.argsort(X) + 1  # Mantendo os índices da ordenação acrescidos de 1 para dar match com a matris S

    M_t = np.zeros((len(X_indice), 2 ** n))  # Matriz de zeros para ser substituida pelos valores de  P_new

    # Etapa 3) Cálculo da Integral de Choquet
    for i in range(0, len(X)):  # Roda para todas as linhas (alternativas)
        for j in range(1, n + 1):  # Interaçoes que acontecem nas colunas
            X_indice_aux = np.pad(np.sort(X_indice[i][j - 1:n]), ((0, j - 1)), 'constant',
                                  constant_values=0)  # Obtendo p/ cada interação os índices ordenados acrescidos de zeros
            M_t[i][np.flatnonzero(np.all(S == X_indice_aux, 1))] = X_new[i][j] - X_new[i][
                j - 1]  # Substituindo onde da Match entre S e P_indice_aux na matrix M os valores de P_new

    global_eval = np.dot(M_t, capacidade)  # Valor da Integral de Choquet.
    print('Integral de Choquet:', global_eval)
    return global_eval


# Funções para calcular a transformação da capacidade para os índices de interação (para todas as coalizões)
def ncr(q, p):
    r = min(q, q - p)
    numer = reduce(op.mul, range(q, q - p, -1), 1)
    denom = reduce(op.mul, range(1, p + 1), 1)
    return numer // denom  # or / in Python 2


def indice_interacao(capacidade):
    n_coal = len(capacidade)
    n = int(math.log(n_coal, 2))
    p = np.arange(1, n + 1)

    B = bernoulli(n)  # Números de Bernoulli
    Brq = np.zeros((n + 1, n + 1))  # Matriz de suporte, inicialmente zerada

    for r in range(0, n + 1):
        for q in range(0, r + 1):
            a = sum([ncr(q, p) * B[r - p] for p in range(0, r + 1)])
            Brq[q][r] = a

    # Construindo, definitivamente, a Matriz de Transformação T
    # 1) Valores de r, na Matriz de Transformação T:
    r_vector = []
    for i in range(1, n + 1):
        teste_x = np.array(list(itertools.combinations(p, i)))
        for i in range(0, len(teste_x)):
            r_vector += [len(teste_x[i])]
    r = np.pad(r_vector, ((1, 0)), 'constant', constant_values=0)

    # 2) Valores de q, na Matriz de Transformação T:
    S_1 = [np.array(list(itertools.combinations(p, r))) for r in range(1, n + 1)]
    q_vector = []
    for i in range(0, n):
        for j in range(0, len(S_1[i])):
            for k in range(0, n):
                for l in range(0, len(S_1[k])):
                    q_vector += [sum(np.in1d(S_1[k], S_1[i][j]).reshape(S_1[k].shape)[l])]

    q_final = np.reshape(q_vector, (2 ** n - 1, 2 ** n - 1))
    q = np.pad(q_final, ((1, 0), (1, 0)), 'constant', constant_values=0)

    # 3) Subistituindo os valores corretos nas posições corretas na Matriz de Transformação:
    T = np.zeros((2 ** n, 2 ** n))
    for i in range(0, 2 ** n):
        for j in range(0, 2 ** n):
            T[i][j] = Brq[q[i][j]][r[j]]

    inter_indices = np.dot(np.linalg.inv(T), capacidade)

    return n, inter_indices

In [60]:
def norm_binario(frame, crit):
    # Matriz de decisão 
    decisao = np.array(frame[frame.columns[1:]]).T
    
    X = []
    for i, m in zip(decisao, crit):
        # Armazenar o valores obtidos
        c = []
        # Encontrar valor de max e min
        minimo = min(i)
        maximo = max(i)
        if m: # Se o critério for max
            for j in i:
                c.append((j-minimo)/(maximo-minimo))
        else:  # Se o critério for Min
            for j in i:
                c.append((maximo-j)/(maximo-minimo))
        X.append(c)
    X = np.transpose(X)
    return X

In [61]:
def saw(crit, peso, frame, norm=True):
    # Verificar se o código já foi utilizado
    primeira_col = frame.columns[0]
    for i in frame.columns:
        if i == 'Ordenamento_saw':
            frame.drop(columns=['Ordenamento_saw'], inplace=True)
        if i == 'Ordenamento_topsis':
            frame.drop(columns=['Ordenamento_topsis'], inplace=True)
        else:
            pass
    if norm:
        # Matriz de Decisão: X
        X = norm_binario(frame=frame, crit=crit)
    else:
        X = np.array(frame[frame.columns[1:]])

    v = []
    # Passo 1
    for k in X:
        v.append(round(sum(k*peso), 2))
    
    # Passo 2:
    frame['Ordenamento_saw'] = v
    final = list(frame.sort_values(by='Ordenamento_saw', ascending=False)[primeira_col])
    ordem = 1
    for x in final:
        print(f'{ordem}°', x)
        ordem += 1
    return frame

# Atividade 1

In [16]:
u1 = 0
u2 = 0.4
u3 = 0.6
u12 = u1 + u2
u13 = u1 + u3
u23 = u2 + u3
u123 = u1 + u2 + u3

In [17]:
m1 = u1 * 1
m2 = u2 * 1
m3 = u3 * 1
m12 = u12 * 1.3
m13 = u13 * 0.8
m23 = u23 * 1.2
m123 = u123 * 1 

In [20]:
# Definir a capacidade em ordem lexicográfica
capacidade=[0, m1, m2, m3, m12, m13, m23, m123]

In [21]:
# Definir a matriz de dados (já normalizada)
X = [[8, 7, 4],[8, 4, 7],[4, 7, 4], [4, 4, 7]]

# Cálculo da intergal de Choquet para todas as alternativas
eval = intChoquet(X,capacidade)

# Cálculo dos índices de interação
n, ind_iter = indice_interacao(capacidade)

# Print do vetor com todos os índices de interação
print('Vetor com todos os índices de interação:', ind_iter)

# Print dos valores de Shapley
print('Valores de Shapley:', ind_iter[1:n+1])

# Print dos índices de interação (pares de critérios)
print('Índices de interação:', ind_iter[n+1:int(n+1+(n*(n-1)/2))])

Integral de Choquet: [5.56 5.44 5.2  5.8 ]
Vetor com todos os índices de interação: [ 0.51666667 -0.06666667  0.49333333  0.57333333  0.02       -0.22
  0.1        -0.2       ]
Valores de Shapley: [-0.06666667  0.49333333  0.57333333]
Índices de interação: [ 0.02 -0.22  0.1 ]


# Atividade 2

## Caso 1

In [22]:
u1 = 0.6
u2 = 0.4
u12 = u1 + u2

In [23]:
m1 = u1 * 1
m2 = u2 * 1
m12 = u12 * 1.3

In [24]:
# Definir a matriz de dados (já normalizada)
X = [[2, 9],[9, 2],[7, 7]]

# Definir a capacidade em ordem lexicográfica
capacidade=[0, m1, m2, m12]

# Cálculo da intergal de Choquet para todas as alternativas
eval = intChoquet(X,capacidade)

# Cálculo dos índices de interação
n, ind_iter = indice_interacao(capacidade)

# Print do vetor com todos os índices de interação
print('Vetor com todos os índices de interação:', ind_iter)

# Print dos valores de Shapley
print('Valores de Shapley:', ind_iter[1:n+1])

# Print dos índices de interação (pares de critérios)
print('Índices de interação:', ind_iter[n+1:int(n+1+(n*(n-1)/2))])

Integral de Choquet: [5.4 6.8 9.1]
Vetor com todos os índices de interação: [0.6  0.75 0.55 0.3 ]
Valores de Shapley: [0.75 0.55]
Índices de interação: [0.3]


## Caso 2

In [25]:
u1 = 0.6
u2 = 0.4
u12 = u1 + u2

In [26]:
m1 = u1 * 1
m2 = u2 * 1
m12 = u12 * 1.3

In [28]:
# Definir a matriz de dados (já normalizada)
X = [[10, 3],[3, 10],[6, 6]]

# Definir a capacidade em ordem lexicográfica
capacidade=[0, m1, m2, m12]

# Cálculo da intergal de Choquet para todas as alternativas
eval = intChoquet(X,capacidade)

# Cálculo dos índices de interação
n, ind_iter = indice_interacao(capacidade)

# Print do vetor com todos os índices de interação
print('Vetor com todos os índices de interação:', ind_iter)

# Print dos valores de Shapley
print('Valores de Shapley:', ind_iter[1:n+1])

# Print dos índices de interação (pares de critérios)
print('Índices de interação:', ind_iter[n+1:int(n+1+(n*(n-1)/2))])

Integral de Choquet: [8.1 6.7 7.8]
Vetor com todos os índices de interação: [0.6  0.75 0.55 0.3 ]
Valores de Shapley: [0.75 0.55]
Índices de interação: [0.3]


# Atividade 3

In [41]:
u1 = 0.3
u2 = 0.1
u3 = 0.2
u4 = 0.4
u12 = u1 + u2
u13 = u1 + u3
u14 = u1 + u4
u23 = u2 + u3
u24 = u2 + u4
u34 = u3 + u4
u123 = u1 + u2 + u3
u124 = u1 + u2 + u4
u134 = u1 + u3 + u4
u234 = u2 + u3 + u4

In [42]:
m1 = u1 * 1
m2 = u2 * 1  
m3 = u3 * 1
m4 = u4 * 1
m12 = u12 * 1.1
m13 = u13 * 1
m14 = u14 * 1.4
m23 = u23 * 1
m24 = u24 * 1.2
m34 = u34 * 1
m123 = u123 * 1.1
m124 = u124 * 1.2
m134 = u134 * 1
m234 = u234 * 1

In [62]:
# Definir a matriz de dados
X =[[6.55, 14, 569, 3],
    [2.35, 13, 230, 5],
    [8.49, 17, 727, 7],
    [9.33, 18, 822, 9],
    [6.78, 11, 578, 7],
    [7.57, 14, 911, 5],
    [7.43, 14, 665, 4],
    [3.92, 16, 429, 5],
    [6.55, 17, 632, 3],
    [1.71, 17, 282, 8]]

In [63]:
# Definir a capacidade em ordem lexicográfica
capacidade=[0, m1, m2, m3, m4, m12, m13, m14, m23, m24, m34, m123, m124, m134, m234, 1]

# Cálculo da intergal de Choquet para todas as alternativas
eval = intChoquet(X,capacidade)

# Cálculo dos índices de interação
n, ind_iter = indice_interacao(capacidade)

# Print do vetor com todos os índices de interação
print('Vetor com todos os índices de interação:', ind_iter)

# Print dos valores de Shapley
print('Valores de Shapley:', ind_iter[1:n+1])

# Print dos índices de interação (pares de critérios)
print('Índices de interação:', ind_iter[n+1:int(n+1+(n*(n-1)/2))])

Integral de Choquet: [118.578   50.005  152.5364 172.6188 121.534  188.0252 138.4348  90.576
 131.478   61.813 ]
Vetor com todos os índices de interação: [ 0.525       0.33666667  0.10666667  0.13        0.42666667 -0.01333333
 -0.06333333  0.07666667  0.02666667 -0.01333333 -0.12333333  0.12
 -0.16       -0.18        0.          0.2       ]
Valores de Shapley: [0.33666667 0.10666667 0.13       0.42666667]
Índices de interação: [-0.01333333 -0.06333333  0.07666667  0.02666667 -0.01333333 -0.12333333]


In [105]:
crit_max = [False, False, True, True]
w = [0.3, 0.2, 0.1, 0.4]

df = pd.DataFrame(data=X, columns=['Custo', 'Tempo', 'B.', 'Impacto'])

In [106]:
df

,Custo,Tempo,B.,Impacto
0,6.55,14,569,3
1,2.35,13,230,5
2,8.49,17,727,7
3,9.33,18,822,9
4,6.78,11,578,7
5,7.57,14,911,5
6,7.43,14,665,4
7,3.92,16,429,5
8,6.55,17,632,3
9,1.71,17,282,8


In [107]:
n = 1
nome = []
for _ in range(len(X)):
    nome.append(f'Alternativa{n}')
    n +=1
df['Alternativas'] = nome
df = df[['Alternativas', 'Custo', 'Tempo', 'B.', 'Impacto']]

In [108]:
saida = saw(crit=crit_max, peso=w, frame=df)
saida

1° Alternativa10
2° Alternativa5
3° Alternativa2
4° Alternativa4
5° Alternativa8
6° Alternativa6
7° Alternativa3
8° Alternativa7
9° Alternativa1
10° Alternativa9


<ipython-input-61-527d4fc05242>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  frame['Ordenamento_saw'] = v


,Alternativas,Custo,Tempo,B.,Impacto,Ordenamento_saw
0,Alternativa1,6.55,14,569,3,0.27
1,Alternativa2,2.35,13,230,5,0.55
2,Alternativa3,8.49,17,727,7,0.40
3,Alternativa4,9.33,18,822,9,0.49
4,Alternativa5,6.78,11,578,7,0.62
5,Alternativa6,7.57,14,911,5,0.42
6,Alternativa7,7.43,14,665,4,0.32
7,Alternativa8,3.92,16,429,5,0.43
8,Alternativa9,6.55,17,632,3,0.20
9,Alternativa10,1.71,17,282,8,0.67
